In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import gradio

In [8]:
# Device configuration
device = torch.device('cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='../../data',train=False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch [1/2], Step [100/600], Loss: 0.4317
Epoch [1/2], Step [200/600], Loss: 0.2267
Epoch [1/2], Step [300/600], Loss: 0.2052
Epoch [1/2], Step [400/600], Loss: 0.1179
Epoch [1/2], Step [500/600], Loss: 0.1108
Epoch [1/2], Step [600/600], Loss: 0.1830
Epoch [2/2], Step [100/600], Loss: 0.0972
Epoch [2/2], Step [200/600], Loss: 0.0662
Epoch [2/2], Step [300/600], Loss: 0.1487
Epoch [2/2], Step [400/600], Loss: 0.0640
Epoch [2/2], Step [500/600], Loss: 0.0425
Epoch [2/2], Step [600/600], Loss: 0.0979


In [10]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.04 %


In [39]:
value = torch.from_numpy(images.numpy())
print(value.dtype)
value = torch.autograd.Variable(value)
print(value.dtype)
prediction = model(value)

torch.float64
torch.float64


RuntimeError: Expected object of type torch.FloatTensor but found type torch.DoubleTensor for argument #4 'mat1'

In [38]:
images.numpy().astype('float64').dtype

dtype('float64')

In [21]:
prediction.data.numpy().shape

(100, 10)

In [42]:
prediction.data.numpy()

array([[-2.94313431e+00, -1.81460023e+00, -2.08448991e-01,
        -2.29123878e+00, -2.91417217e+00, -7.30904102e-01,
        -1.85286796e+00, -8.89607048e+00,  3.85826755e+00,
        -5.70444298e+00],
       [-5.27852488e+00, -9.87475681e+00, -3.23101878e+00,
        -3.27192068e+00,  2.99915814e+00, -4.19678402e+00,
        -6.34950256e+00, -4.51865005e+00,  7.18662143e-01,
         4.91613436e+00],
       [ 5.31619835e+00, -4.94643354e+00, -7.60741353e-01,
        -3.37821364e+00, -2.58448744e+00, -1.16258490e+00,
        -2.44758511e+00, -2.42502451e+00, -2.97429585e+00,
        -8.71329665e-01],
       [-6.26879740e+00,  5.35215139e+00, -1.39423239e+00,
        -3.57356954e+00, -1.04397392e+00, -6.51621342e+00,
        -5.03530502e+00, -3.36044490e-01, -1.06999171e+00,
        -5.35540390e+00],
       [ 5.72689712e-01, -4.73341894e+00,  9.67390776e-01,
        -7.11784005e-01, -2.87459540e+00, -3.85147333e-03,
        -1.63910186e+00, -3.20800948e+00, -1.86211896e+00,
        -5.

float32
torch.float32
float32
torch.float32
float32
torch.float32


In [40]:
inp = gradio.inputs.Sketchpad(flatten=True, scale=1/255, dtype='float32')
io = gradio.Interface(inputs=inp, outputs="label", model_type="pytorch", model=model)

In [41]:
io.launch()

No validation samples for this interface... skipping validation.
NOTE: Gradio is in beta stage, please report all bugs to: a12d@stanford.edu
Model is running locally at: http://localhost:7874/interface.html
To create a public link, set `share=True` in the argument to `launch()`


(<gradio.networking.serve_files_in_background.<locals>.HTTPServer at 0x1450966be48>,
 'http://localhost:7874/',
 None)

float32
torch.float32


In [ ]:
model